In [6]:
import findspark
findspark.init('/usr/local/Cellar/apache-spark/3.0.1/libexec')

import pyspark
from pyspark.sql.session import SparkSession
spark = SparkSession.builder.appName("spark test").getOrCreate()

In [9]:
from pyspark.sql.types import *
import pandas as pd
pd.options.display.max_columns = None
from pyspark.sql.functions import *
import pandas as pd
import numpy as np
from pyspark.sql.types import *


In [10]:
timeslot_2G = [StructField("PERIOD_START_TIME", StringType(), True),
                StructField("BSC name", StringType(), True),
                StructField("BCF name", StringType(), True),
                StructField("BTS name", StringType(), True),
                StructField("Timeslot Util", DoubleType(), True)]

timeslot2GSchema = StructType(timeslot_2G)

In [11]:
df_timeslot_2G = spark.read.format("csv") \
    .option("header", "true") \
    .option("delimiter", ";") \
    .schema(timeslot2GSchema) \
    .load("Timeslot occupancy 2G_Hourly.csv")

In [12]:
from datetime import datetime


In [13]:
df_timeslot_2G.show()

+-------------------+----------------+--------------------+--------------------+-------------+
|  PERIOD_START_TIME|        BSC name|            BCF name|            BTS name|Timeslot Util|
+-------------------+----------------+--------------------+--------------------+-------------+
|09.19.2020 16:00:00|BSC_ABENGOUROU_1|  OCI0005_ABENGOUROU|OCI0005_ABENGOURO...|        36.07|
|09.19.2020 16:00:00|BSC_ABENGOUROU_1|  OCI0005_ABENGOUROU|OCI0005_ABENGOURO...|        46.43|
|09.19.2020 16:00:00|BSC_ABENGOUROU_1|  OCI0005_ABENGOUROU|OCI0005_ABENGOURO...|        30.96|
|09.19.2020 16:00:00|BSC_ABENGOUROU_1|  OCI0005_ABENGOUROU|OCI0005_ABENGOURO...|        10.11|
|09.19.2020 16:00:00|BSC_ABENGOUROU_1|  OCI0005_ABENGOUROU|OCI0005_ABENGOURO...|        22.11|
|09.19.2020 16:00:00|BSC_ABENGOUROU_1|  OCI0005_ABENGOUROU|OCI0005_ABENGOURO...|        23.06|
|09.19.2020 16:00:00|BSC_ABENGOUROU_1|OCI0006_S_ABENGOU...|OCI0006_S_ABENGOU...|        46.21|
|09.19.2020 16:00:00|BSC_ABENGOUROU_1|OCI0006_S_AB

In [15]:
## Convert to timestap
df_timeslot_2G = df_timeslot_2G.withColumn('PERIOD_START_TIME',to_timestamp(df_timeslot_2G.PERIOD_START_TIME, 'MM.dd.yyyy HH:mm:ss').cast("timestamp"))

In [16]:
oldColumns = df_timeslot_2G.schema.names
oldColumns

['PERIOD_START_TIME', 'BSC name', 'BCF name', 'BTS name', 'Timeslot Util']

In [17]:
from functools import reduce

oldColumns = df_timeslot_2G.schema.names
newColumns = ['date', 'BSC_name', 'BCF_name', 'BTS_name', 'timeslot_util']

df_timeslot_2G = reduce(lambda df_timeslot_2G, idx: df_timeslot_2G.withColumnRenamed(oldColumns[idx], newColumns[idx]), range(len(oldColumns)), df_timeslot_2G)


In [111]:
## Save as parquet
#df_timeslot_2G.write.parquet("timeslot2G.parquet")

In [18]:
df_timeslot_2G.show()

+-------------------+----------------+--------------------+--------------------+-------------+
|               date|        BSC_name|            BCF_name|            BTS_name|timeslot_util|
+-------------------+----------------+--------------------+--------------------+-------------+
|2020-09-19 16:00:00|BSC_ABENGOUROU_1|  OCI0005_ABENGOUROU|OCI0005_ABENGOURO...|        36.07|
|2020-09-19 16:00:00|BSC_ABENGOUROU_1|  OCI0005_ABENGOUROU|OCI0005_ABENGOURO...|        46.43|
|2020-09-19 16:00:00|BSC_ABENGOUROU_1|  OCI0005_ABENGOUROU|OCI0005_ABENGOURO...|        30.96|
|2020-09-19 16:00:00|BSC_ABENGOUROU_1|  OCI0005_ABENGOUROU|OCI0005_ABENGOURO...|        10.11|
|2020-09-19 16:00:00|BSC_ABENGOUROU_1|  OCI0005_ABENGOUROU|OCI0005_ABENGOURO...|        22.11|
|2020-09-19 16:00:00|BSC_ABENGOUROU_1|  OCI0005_ABENGOUROU|OCI0005_ABENGOURO...|        23.06|
|2020-09-19 16:00:00|BSC_ABENGOUROU_1|OCI0006_S_ABENGOU...|OCI0006_S_ABENGOU...|        46.21|
|2020-09-19 16:00:00|BSC_ABENGOUROU_1|OCI0006_S_AB

In [19]:
## Save as parquet
df_timeslot_2G.groupBy(["date"]).count().show(10, False)

+-------------------+-----+
|date               |count|
+-------------------+-----+
|2020-10-05 10:00:00|1208 |
|2020-10-08 03:00:00|1208 |
|2020-10-10 15:00:00|1208 |
|2020-10-16 11:00:00|1208 |
|2020-09-23 19:00:00|2181 |
|2020-09-29 13:00:00|1199 |
|2020-10-18 01:00:00|1208 |
|2020-10-07 01:00:00|1208 |
|2020-10-12 09:00:00|1208 |
|2020-10-18 21:00:00|1208 |
+-------------------+-----+
only showing top 10 rows



In [20]:
df_timeslot_2G.show()

+-------------------+----------------+--------------------+--------------------+-------------+
|               date|        BSC_name|            BCF_name|            BTS_name|timeslot_util|
+-------------------+----------------+--------------------+--------------------+-------------+
|2020-09-19 16:00:00|BSC_ABENGOUROU_1|  OCI0005_ABENGOUROU|OCI0005_ABENGOURO...|        36.07|
|2020-09-19 16:00:00|BSC_ABENGOUROU_1|  OCI0005_ABENGOUROU|OCI0005_ABENGOURO...|        46.43|
|2020-09-19 16:00:00|BSC_ABENGOUROU_1|  OCI0005_ABENGOUROU|OCI0005_ABENGOURO...|        30.96|
|2020-09-19 16:00:00|BSC_ABENGOUROU_1|  OCI0005_ABENGOUROU|OCI0005_ABENGOURO...|        10.11|
|2020-09-19 16:00:00|BSC_ABENGOUROU_1|  OCI0005_ABENGOUROU|OCI0005_ABENGOURO...|        22.11|
|2020-09-19 16:00:00|BSC_ABENGOUROU_1|  OCI0005_ABENGOUROU|OCI0005_ABENGOURO...|        23.06|
|2020-09-19 16:00:00|BSC_ABENGOUROU_1|OCI0006_S_ABENGOU...|OCI0006_S_ABENGOU...|        46.21|
|2020-09-19 16:00:00|BSC_ABENGOUROU_1|OCI0006_S_AB

In [21]:
from pyspark.sql.functions import hour,month,dayofmonth,weekofyear,year,date_format
df_timeslot_2G = df_timeslot_2G.withColumn('hour',hour(df_timeslot_2G.date))
df_timeslot_2G = df_timeslot_2G.withColumn('month',month(df_timeslot_2G.date))
df_timeslot_2G = df_timeslot_2G.withColumn('dayofmonth',dayofmonth(df_timeslot_2G.date))
df_timeslot_2G = df_timeslot_2G.withColumn('weekofyear',weekofyear(df_timeslot_2G.date))
df_timeslot_2G = df_timeslot_2G.withColumn('year',year(df_timeslot_2G.date))
df_timeslot_2G = df_timeslot_2G.withColumn('weekday',date_format(df_timeslot_2G.date,'E'))
df_timeslot_2G.show()


+-------------------+----------------+--------------------+--------------------+-------------+----+-----+----------+----------+----+
|               date|        BSC_name|            BCF_name|            BTS_name|timeslot_util|hour|month|dayofmonth|weekofyear|year|
+-------------------+----------------+--------------------+--------------------+-------------+----+-----+----------+----------+----+
|2020-09-19 16:00:00|BSC_ABENGOUROU_1|  OCI0005_ABENGOUROU|OCI0005_ABENGOURO...|        36.07|  16|    9|        19|        38|2020|
|2020-09-19 16:00:00|BSC_ABENGOUROU_1|  OCI0005_ABENGOUROU|OCI0005_ABENGOURO...|        46.43|  16|    9|        19|        38|2020|
|2020-09-19 16:00:00|BSC_ABENGOUROU_1|  OCI0005_ABENGOUROU|OCI0005_ABENGOURO...|        30.96|  16|    9|        19|        38|2020|
|2020-09-19 16:00:00|BSC_ABENGOUROU_1|  OCI0005_ABENGOUROU|OCI0005_ABENGOURO...|        10.11|  16|    9|        19|        38|2020|
|2020-09-19 16:00:00|BSC_ABENGOUROU_1|  OCI0005_ABENGOUROU|OCI0005_AB

In [31]:
## Add period to the dataframe
def getPeriodo(month, year):
    if int(month) <10:
        month = "0" + str(month)
    return [str(year) + str(month)]
getPeriodo = udf(getPeriodo, ArrayType(StringType()))

df_timeslot_2G = df_timeslot_2G.withColumn("period",*[getPeriodo('month', 'year')[i] for i in range(0, 1)])
df_timeslot_2G.show()

+-------------------+----------------+--------------------+--------------------+-------------+----+-----+----------+----------+----+------+
|               date|        BSC_name|            BCF_name|            BTS_name|timeslot_util|hour|month|dayofmonth|weekofyear|year|period|
+-------------------+----------------+--------------------+--------------------+-------------+----+-----+----------+----------+----+------+
|2020-09-19 16:00:00|BSC_ABENGOUROU_1|  OCI0005_ABENGOUROU|OCI0005_ABENGOURO...|        36.07|  16|    9|        19|        38|2020|202009|
|2020-09-19 16:00:00|BSC_ABENGOUROU_1|  OCI0005_ABENGOUROU|OCI0005_ABENGOURO...|        46.43|  16|    9|        19|        38|2020|202009|
|2020-09-19 16:00:00|BSC_ABENGOUROU_1|  OCI0005_ABENGOUROU|OCI0005_ABENGOURO...|        30.96|  16|    9|        19|        38|2020|202009|
|2020-09-19 16:00:00|BSC_ABENGOUROU_1|  OCI0005_ABENGOUROU|OCI0005_ABENGOURO...|        10.11|  16|    9|        19|        38|2020|202009|
|2020-09-19 16:00:00

In [32]:
df_timeslot_2G.select('hour', 'month', *[getPeriodo('month', 'year')[i] for i in range(0, 1)]) \
                  .toDF('hour', 'month', 'period').show()

+----+-----+------+
|hour|month|period|
+----+-----+------+
|  16|    9|202009|
|  16|    9|202009|
|  16|    9|202009|
|  16|    9|202009|
|  16|    9|202009|
|  16|    9|202009|
|  16|    9|202009|
|  16|    9|202009|
|  16|    9|202009|
|  16|    9|202009|
|  16|    9|202009|
|  16|    9|202009|
|  16|    9|202009|
|  16|    9|202009|
|  16|    9|202009|
|  16|    9|202009|
|  16|    9|202009|
|  16|    9|202009|
|  16|    9|202009|
|  16|    9|202009|
+----+-----+------+
only showing top 20 rows



In [43]:
df_timeslot_2G = df_timeslot_2G.withColumn("week_period",*[getPeriodo('weekofyear', 'year')[i] for i in range(0, 1)])
df_timeslot_2G.show(3)

+-------------------+----------------+------------------+--------------------+-------------+----+-----+----------+----------+----+------+-----------+-------+
|               date|        BSC_name|          BCF_name|            BTS_name|timeslot_util|hour|month|dayofmonth|weekofyear|year|period|week_period|weekday|
+-------------------+----------------+------------------+--------------------+-------------+----+-----+----------+----------+----+------+-----------+-------+
|2020-09-19 16:00:00|BSC_ABENGOUROU_1|OCI0005_ABENGOUROU|OCI0005_ABENGOURO...|        36.07|  16|    9|        19|        38|2020|202009|     202038|    Sat|
|2020-09-19 16:00:00|BSC_ABENGOUROU_1|OCI0005_ABENGOUROU|OCI0005_ABENGOURO...|        46.43|  16|    9|        19|        38|2020|202009|     202038|    Sat|
|2020-09-19 16:00:00|BSC_ABENGOUROU_1|OCI0005_ABENGOUROU|OCI0005_ABENGOURO...|        30.96|  16|    9|        19|        38|2020|202009|     202038|    Sat|
+-------------------+----------------+--------------

In [64]:

from pyspark.sql import functions as F

df_timeslot_2G_agg = df_timeslot_2G.groupBy(['BSC_name','week_period']).agg(F.min('timeslot_util'), F.max('timeslot_util'), F.mean('timeslot_util'))

## Change name columns
oldColumns = ['BSC_name', 'week_period', 'min(timeslot_util)', 'max(timeslot_util)','avg(timeslot_util)']
newColumns = ['BSC_name', 'week_period', 'min_time_slot_all', 'max_time_slot_all','avg_time_slot_all']
df_timeslot_2G_agg = reduce(lambda df_timeslot_2G_agg, idx: df_timeslot_2G_agg.withColumnRenamed(oldColumns[idx], newColumns[idx]), range(len(oldColumns)), df_timeslot_2G_agg)

df_timeslot_2G_agg.show(3)

+----------------+-----------+-----------------+-----------------+------------------+
|        BSC_name|week_period|min_time_slot_all|max_time_slot_all| avg_time_slot_all|
+----------------+-----------+-----------------+-----------------+------------------+
|BSC_ABENGOUROU_1|     202040|              0.0|             98.7|21.636957161106604|
|BSC_ABENGOUROU_1|     202042|              0.0|            99.95|21.620607014743126|
|BSC_ABENGOUROU_1|     202039|              0.0|            97.55|21.195860754554857|
+----------------+-----------+-----------------+-----------------+------------------+
only showing top 3 rows



In [65]:
## Get info of weekend
df_timeslot_2G_weekend = df_timeslot_2G.filter((df_timeslot_2G.weekday.isin(['Sat','Sun','Fri'])))
df_timeslot_2G_weekend_agg = df_timeslot_2G_weekend.groupBy(['BSC_name','week_period']).agg(F.min('timeslot_util'), F.max('timeslot_util'), F.mean('timeslot_util'))

## Change name columns
oldColumns = ['BSC_name', 'week_period', 'min(timeslot_util)', 'max(timeslot_util)','avg(timeslot_util)']
newColumns = ['BSC_name', 'week_period', 'min_time_slot_weekend', 'max_time_slot_weekend','avg_time_slot_weekend']
df_timeslot_2G_weekend_agg = reduce(lambda df_timeslot_2G_weekend_agg, idx: df_timeslot_2G_weekend_agg.withColumnRenamed(oldColumns[idx], newColumns[idx]), range(len(oldColumns)), df_timeslot_2G_weekend_agg)

df_timeslot_2G_weekend_agg.show()

+----------------+-----------+---------------------+---------------------+---------------------+
|        BSC_name|week_period|min_time_slot_weekend|max_time_slot_weekend|avg_time_slot_weekend|
+----------------+-----------+---------------------+---------------------+---------------------+
|BSC_ABENGOUROU_1|     202040|                  0.0|                 98.7|    22.06136031866641|
|BSC_ABENGOUROU_1|     202042|                  0.0|                99.95|   22.248899696467866|
|BSC_ABENGOUROU_1|     202039|                  0.0|                97.25|   21.406300326820187|
|BSC_ABENGOUROU_1|     202041|                  0.0|                98.54|    20.81176427980175|
|BSC_ABENGOUROU_1|     202038|                  0.0|                96.94|   22.474968906772457|
|    BSC_BOUAKE_1|     202039|                  0.0|                99.13|   12.940757931542676|
|    BSC_BOUAKE_1|     202038|                  0.0|                99.15|   15.095980012690527|
+----------------+-----------+

In [66]:
df_timeslot_2G_not_weekend = df_timeslot_2G.filter((df_timeslot_2G.weekday.isin(['Mon','Tue','Wed','Thu'])))
df_timeslot_2G_not_weekend_agg = df_timeslot_2G_not_weekend.groupBy(['BSC_name','week_period']).agg(F.min('timeslot_util'), F.max('timeslot_util'), F.mean('timeslot_util'))

## Change name columns
oldColumns = ['BSC_name', 'week_period', 'min(timeslot_util)', 'max(timeslot_util)','avg(timeslot_util)']
newColumns = ['BSC_name', 'week_period', 'min_time_slot_not_weekend', 'max_time_slot_not_weekend','avg_time_slot_not_weekend']
df_timeslot_2G_not_weekend_agg = reduce(lambda df_timeslot_2G_not_weekend_agg, idx: df_timeslot_2G_not_weekend_agg.withColumnRenamed(oldColumns[idx], newColumns[idx]), range(len(oldColumns)), df_timeslot_2G_not_weekend_agg)

df_timeslot_2G_not_weekend_agg.show()

+----------------+-----------+-------------------------+-------------------------+-------------------------+
|        BSC_name|week_period|min_time_slot_not_weekend|max_time_slot_not_weekend|avg_time_slot_not_weekend|
+----------------+-----------+-------------------------+-------------------------+-------------------------+
|BSC_ABENGOUROU_1|     202040|                      0.0|                    97.58|        21.31668586669481|
|BSC_ABENGOUROU_1|     202042|                      0.0|                    99.44|       21.149387503449365|
|BSC_ABENGOUROU_1|     202039|                      0.0|                    97.55|        21.03604743883108|
|BSC_ABENGOUROU_1|     202041|                      0.0|                    98.07|        21.62855770557422|
|    BSC_BOUAKE_1|     202039|                      0.0|                    99.42|       14.458623942470487|
|BSC_ABENGOUROU_1|     202043|                      0.0|                     97.1|        16.97109437086095|
+----------------+-

In [69]:
## Merge dataset 

final_df = df_timeslot_2G_agg.join(df_timeslot_2G_weekend_agg,
                                  on = ['BSC_name','week_period'],
                                  how = 'left')
final_df = final_df.join(df_timeslot_2G_not_weekend_agg,
                         on = ['BSC_name','week_period'],
                                  how = 'left')

In [70]:
final_df.show()

+----------------+-----------+-----------------+-----------------+------------------+---------------------+---------------------+---------------------+-------------------------+-------------------------+-------------------------+
|        BSC_name|week_period|min_time_slot_all|max_time_slot_all| avg_time_slot_all|min_time_slot_weekend|max_time_slot_weekend|avg_time_slot_weekend|min_time_slot_not_weekend|max_time_slot_not_weekend|avg_time_slot_not_weekend|
+----------------+-----------+-----------------+-----------------+------------------+---------------------+---------------------+---------------------+-------------------------+-------------------------+-------------------------+
|BSC_ABENGOUROU_1|     202040|              0.0|             98.7|21.636957161106604|                  0.0|                 98.7|    22.06136031866641|                      0.0|                    97.58|        21.31668586669481|
|BSC_ABENGOUROU_1|     202042|              0.0|            99.95|21.62060701474